In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from keras.utils import to_categorical
from matplotlib import pyplot
from tensorflow.keras import regularizers
import tensorflow as tfs
 

In [2]:
X_train = load('trainX.npy')
X_test = load('testX.npy')
y_train = load('trainy.npy')
y_test = load('testy.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(44339, 128, 6) (44339, 20) (4936, 128, 6) (4936, 20)


In [3]:
# reshape data into time steps of sub-sequences
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
n_steps, n_length = 4, 32
X_train = X_train.reshape((X_train.shape[0], n_steps, n_length, n_features))
X_test = X_test.reshape((X_test.shape[0], n_steps, n_length, n_features))

In [4]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [5]:
model2 = Sequential()
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu'), input_shape=(None,n_length,n_features)))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu')))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu')))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu')))
model2.add(TimeDistributed(Dropout(0.5)))
model2.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model2.add(TimeDistributed(Flatten()))
model2.add(LSTM(100))
model2.add(Dropout(0.5))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(n_outputs, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, None, 28, 64)      1984      
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 24, 64)      20544     
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 20, 64)      20544     
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 16, 64)      20544     
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 16, 64)      0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, None, 8, 64)       0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, None, 512)         0

In [6]:
# fit network
history = model2.fit(X_training, y_training, epochs=10, verbose=False, validation_data=(X_validation, y_validation), batch_size=100)

In [7]:
# evaluate model
loss, accuracy = model2.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model2.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9842
Testing Accuracy: 0.9293
